In [1]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px

In [2]:
df = pd.read_csv("raw_ksi_data.csv")

In [3]:
df.shape

(18957, 54)

In [4]:
df.sample(2)

,OBJECTID,INDEX,ACCNUM,DATE,TIME,STREET1,STREET2,OFFSET,ROAD_CLASS,DISTRICT,...,REDLIGHT,ALCOHOL,DISABILITY,HOOD_158,NEIGHBOURHOOD_158,HOOD_140,NEIGHBOURHOOD_140,DIVISION,x,y
3397,3398,5367386,1041737.0,5/26/2008 8:00:00 AM,1125,1835 YONGE ST,NaN,NaN,Major Arterial,Toronto and East York,...,NaN,NaN,NaN,174,South Eglinton-Davisville,104,Mount Pleasant West (104),D53,629230.3922,4.839429e+06
5672,5673,6128982,1162673.0,3/15/2010 8:00:00 AM,2030,BURNHAMTHORPE RD,MARTIN GROVE RD,NaN,Major Arterial,Etobicoke York,...,NaN,NaN,NaN,158,Islington,14,Islington-City Centre West (14),D22,617018.2564,4.833894e+06


In [5]:
df.drop(['ACCNUM','OBJECTID', 'HOOD_140', 'NEIGHBOURHOOD_140', 'OFFSET', 'ROAD_CLASS', 'ACCLOC', 'INVAGE', 'INITDIR'], axis=1, inplace=True)

In [6]:
df.drop(['VEHTYPE', 'MANOEUVER', 'DRIVACT', 'DRIVCOND', 'PEDTYPE', 'PEDACT', 'CYCLISTYPE', 'CYCACT', 'CYCCOND', 'PEDESTRIAN', "TRSN_CITY_VEH"], axis=1, inplace=True)

In [7]:
df.drop(['EMERG_VEH', 'PASSENGER', 'REDLIGHT', 'DISABILITY', 'DIVISION', 'SPEEDING', 'PEDCOND', 'AG_DRIV','x', 'y' ], axis=1, inplace=True)

In [8]:
df.drop(['CYCLIST', 'IMPACTYPE', 'INVTYPE', 'INDEX', 'TRAFFCTL', 'AUTOMOBILE', 'MOTORCYCLE', 'TRUCK', 'ALCOHOL'], axis=1, inplace=True)

In [9]:
df.shape

(18957, 15)

In [10]:
df.sample(2)

,DATE,TIME,STREET1,STREET2,DISTRICT,LATITUDE,LONGITUDE,VISIBILITY,LIGHT,RDSFCOND,ACCLASS,INJURY,FATAL_NO,HOOD_158,NEIGHBOURHOOD_158
7661,11/16/2011 10:00:00 AM,955,F G GARDINER ExP E,PARK LAWN RD,Etobicoke York,43.626045,-79.48739,Clear,Daylight,Dry,Non-Fatal Injury,Major,NaN,161,Humber Bay Shores
2555,9/14/2007 8:00:00 AM,1913,MANITOBA ST,WESLEY ST,Etobicoke York,43.622245,-79.49749,Rain,Dawn,Wet,Non-Fatal Injury,Major,NaN,160,Mimico-Queensway


In [11]:
df['TIME'] = df['TIME'].astype(str).str.zfill(4)
df['DATE'] = pd.to_datetime(df['DATE'], format='%m/%d/%Y %I:%M:%S %p')

# Extract DOW and HOUR and YEAR
df['DOW'] = df['DATE'].dt.day_name() 
df['HOUR'] = df['TIME'].str[:2].astype(int)
df['YEAR'] = df['DATE'].dt.year

In [12]:
df.sample(3)

,DATE,TIME,STREET1,STREET2,DISTRICT,LATITUDE,LONGITUDE,VISIBILITY,LIGHT,RDSFCOND,ACCLASS,INJURY,FATAL_NO,HOOD_158,NEIGHBOURHOOD_158,DOW,HOUR,YEAR
14272,2018-01-20 10:00:00,2105,MORNINGSIDE AVE,CINEMART DR,Scarborough,43.797301,-79.197437,Clear,"Dark, artificial",Dry,Non-Fatal Injury,NaN,NaN,146,Malvern East,Saturday,21,2018
17534,2022-01-07 10:00:00,0622,DON VALLEY PARKWAY N,LEASIDE BDGE,NaN,43.697722,-79.348224,Clear,"Dark, artificial",Dry,Non-Fatal Injury,NaN,NaN,58,Old East York,Friday,6,2022
10893,2014-08-07 08:00:00,1618,5800 YONGE ST,NaN,North York,43.784381,-79.416683,Clear,Daylight,Dry,Fatal,NaN,NaN,50,Newtonbrook East,Thursday,16,2014


In [13]:
df["INJURY"].unique()

array(['Major', 'Minor', nan, 'Fatal', 'Minimal'], dtype=object)

In [14]:
df["INJURY"].value_counts()

INJURY
Major      6445
Minor      1479
Minimal    1160
Fatal       976
Name: count, dtype: int64

In [15]:
df['INJURY'] = df['INJURY'].fillna('Minimal')

In [16]:
df["ACCLASS"].unique()

array(['Non-Fatal Injury', 'Fatal', 'Property Damage O', nan],
      dtype=object)

In [17]:
df["ACCLASS"].value_counts()

ACCLASS
Non-Fatal Injury     16268
Fatal                 2670
Property Damage O       18
Name: count, dtype: int64

In [18]:
df['ACCLASS'] = df['ACCLASS'].replace('Property Damage O', 'Non-Fatal Injury')
df['ACCLASS'] = df['ACCLASS'].fillna('Non-Fatal Injury')

In [19]:
df['SEVERE_COLLISION'] = (
    (df['INJURY'].isin(['Major', 'Fatal'])) | 
    (df['ACCLASS'] == 'Fatal')
).astype(int)

In [20]:
df['FATAL_NO'] = df['FATAL_NO'].fillna(0)

In [21]:
df["VISIBILITY"].value_counts()

VISIBILITY
Clear                     16373
Rain                       1976
Snow                        356
Other                        98
Fog, Mist, Smoke, Dust       52
Freezing Rain                47
Drifting Snow                21
Strong wind                  10
Name: count, dtype: int64

In [22]:
df['VISIBILITY'] = df['VISIBILITY'].replace({
    'Other': 'Clear',
    'Freezing Rain': 'Rain',
    'Strong wind': 'Rain',
    'Fog, Mist, Smoke, Dust': 'Rain',
    'Drifting Snow': 'Snow'
})

In [23]:
df["VISIBILITY"].value_counts()

VISIBILITY
Clear    16471
Rain      2085
Snow       377
Name: count, dtype: int64

In [24]:
df["LIGHT"].value_counts()

LIGHT
Daylight                10779
Dark                     3746
Dark, artificial         3552
Dusk                      253
Dusk, artificial          253
Daylight, artificial      151
Dawn                      112
Dawn, artificial          101
Other                       6
Name: count, dtype: int64

In [25]:
df['LIGHT'] = df['LIGHT'].replace({
    'Daylight, artificial': 'Daylight',
    'Dark, artificial': 'Dark',
    'Dusk, artificial': 'Dusk',
    'Dawn, artificial': 'Dawn',
    'Other': 'Daylight'
})

In [26]:
df["LIGHT"].value_counts()

LIGHT
Daylight    10936
Dark         7298
Dusk          506
Dawn          213
Name: count, dtype: int64

In [27]:
df.rename(columns={'RDSFCOND': 'ROAD_CONDITION'}, inplace=True)

In [28]:
df["ROAD_CONDITION"].value_counts()

ROAD_CONDITION
Dry                     15231
Wet                      3140
Loose Snow                174
Other                     147
Slush                     102
Ice                        78
Packed Snow                44
Loose Sand or Gravel       11
Spilled liquid              1
Name: count, dtype: int64

In [29]:
df['ROAD_CONDITION'] = df['ROAD_CONDITION'].replace({
    'Spilled liquid': 'Wet',
    'Loose Snow': 'Snow_Ice',
    'Packed Snow': 'Snow_Ice',
    'Slush': 'Snow_Ice',
    'Ice': 'Snow_Ice',
    'Loose Sand or Gravel': 'Dry',
    'Other': 'Dry'
})

In [30]:
df["ROAD_CONDITION"].value_counts()

ROAD_CONDITION
Dry         15389
Wet          3141
Snow_Ice      398
Name: count, dtype: int64

In [31]:
df['DATE'] = pd.to_datetime(df['DATE']).dt.date

In [32]:
df.drop(['TIME'], axis=1, inplace=True)#since hour col, exact time not needed

In [33]:
df.sample(5)

,DATE,STREET1,STREET2,DISTRICT,LATITUDE,LONGITUDE,VISIBILITY,LIGHT,ROAD_CONDITION,ACCLASS,INJURY,FATAL_NO,HOOD_158,NEIGHBOURHOOD_158,DOW,HOUR,YEAR,SEVERE_COLLISION
12842,2016-08-14,STEELES AVE E,REESOR RD,Scarborough,43.848780,-79.197955,Clear,Dark,Dry,Fatal,Minor,0.0,144,Morningside Heights,Sunday,2,2016,1
13165,2016-11-29,BENTWORTH AVE,DUFFERIN ST,North York,43.721705,-79.456401,Clear,Dark,Dry,Non-Fatal Injury,Major,0.0,31,Yorkdale-Glen Park,Tuesday,22,2016,1
58,2006-01-17,KING EDWARD AVE,BURRELL AVE,Toronto and East York,43.690445,-79.311390,Rain,Dark,Wet,Non-Fatal Injury,Minimal,0.0,60,Woodbine-Lumsden,Tuesday,23,2006,0
15085,2018-10-25,WILFRED AVE,OLIVE AVE,North York,43.781642,-79.401424,Clear,Dark,Dry,Non-Fatal Injury,Minimal,0.0,152,East Willowdale,Thursday,20,2018,0
438,2006-05-17,6 HAVENBROOK BLVD,NaN,North York,43.768745,-79.356890,Clear,Daylight,Dry,Non-Fatal Injury,Minimal,0.0,53,Henry Farm,Wednesday,12,2006,0


In [34]:
def categorize_hour(hour):
    if 0 <= hour < 4:
        return 'Late Night'
    elif 4 <= hour < 8:
        return 'Early Morning'
    elif 8 <= hour < 12:
        return 'Morning'
    elif 12 <= hour < 16:
        return 'Afternoon'
    elif 16 <= hour < 20:
        return 'Evening Rush'
    else:
        return 'Night'

In [35]:
df['TIME_OF_DAY'] = df['HOUR'].apply(categorize_hour)

In [36]:
df.shape

(18957, 19)

In [37]:
df.to_csv('cleaned_ksi_collisions.csv', index=False)